In this notebook we will demonstrate how to create your own simple custom models to upload to Waylay BYOML. We will create a function which doubles the inputs.
The dependencies in this environment are:
* waylay-beta
* dill

In [1]:
import numpy as np

In [2]:
class Doubler:
    def predict(self, data):
        return data * 2

In [3]:
X = np.array([1, 2, 3, 4, 5])

In [4]:
doubler = Doubler()
doubler.predict(X)

array([ 2,  4,  6,  8, 10])

In [5]:
from waylay import WaylayClient, RestResponseError
waylay = WaylayClient.from_profile('staging')

In [6]:
MODEL_NAME = 'test_custom_model'

In [8]:
waylay.byoml.model.upload(
        model_name=MODEL_NAME,
        trained_model=doubler, 
        framework='custom',
        description='Simple custom model example'
    )

{'createdOn': '2021-01-04T00:00:00+00:00',
 'description': 'Simple custom model example',
 'framework': 'custom',
 'framework_version': '1.0',
 'message': 'Model successfully uploaded',
 'metadata': {'description': 'Simple custom model example'},
 'name': 'test_custom_model',
 'updatedOn': '2021-01-04T00:00:00+00:00',
 'version': '1'}

In [9]:
waylay.byoml.model.get(MODEL_NAME)

{'createdBy': 'users/f0981778-feed-4d08-823e-dc65e2b8a502',
 'createdOn': '2022-02-15T08:35:09.512000+00:00',
 'description': 'Simple custom model example',
 'framework': 'custom',
 'framework_version': '1.0',
 'metadata': {'description': 'Simple custom model example'},
 'name': 'test_custom_model',
 'ready': False,
 'supported_endpoints': [],
 'updatedBy': 'users/f0981778-feed-4d08-823e-dc65e2b8a502',
 'updatedOn': '2022-02-15T08:35:23.037000+00:00',
 'version': '1'}

In [10]:
waylay.byoml.model.predict(MODEL_NAME, X)

array([ 2,  4,  6,  8, 10])

In [11]:
waylay.byoml.model.remove(MODEL_NAME)

{'message': 'Model test_custom_model: all versions deleted',
 'name': 'test_custom_model',
 'versions': ['0.0.1']}